# <center>***模型融合***<center>

更多代码欢迎传送至：
>https://gitee.com/tanxin12/tanxin

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
names = ['萼片长度', '萼片宽度', '花瓣长度', '花瓣宽度', 'Class']
df1 = pd.read_csv(".\\data\\Iris.txt", names=names)

X = df1.drop("Class", axis=1)
y = df1.Class
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [3]:
filename = ".\\Data\\Boston.txt"
names = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM",
    "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "PRICE"]
df2 = pd.read_csv(filename, sep="\s+", names=names)

X = pd.get_dummies(df2.drop("PRICE", axis=1))
y = df2.PRICE
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.2, random_state=0)

## <center>**Voting——分类模型融合**<center>

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, explained_variance_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [5]:
Model1 = LogisticRegression(max_iter=1000)
Model2 = LinearDiscriminantAnalysis()
Model3 = KNeighborsClassifier()
Model4 = DecisionTreeClassifier()
Model5 = GaussianNB()
Model6 = SVC(probability=True)

Model_plus = VotingClassifier(estimators=[("LR", Model1), ("LDA", Model2),
                            ("KNN", Model3), ("DT", Model4),
                            ("NB", Model5), ("SVM", Model6)],
                            voting="soft")

In [6]:
names = ["LR", "LDA", "KNN", "DT", "NB", "SVM", "Voting"]
model = [Model1, Model2, Model3, Model4, Model5, Model6, Model_plus]
for i in range(0, 7):
    model[i].fit(X1_train, y1_train)
    y1_pred = model[i].predict(X1_test)
    print(names[i], "%.5f" % accuracy_score(y1_pred, y1_test))

LR 1.00000
LDA 1.00000
KNN 0.96667
DT 1.00000
NB 0.96667
SVM 1.00000
Voting 1.00000


## <center>**Stacking——回归模型集成**<center>

In [7]:
from mlxtend.regressor import StackingRegressor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error

In [8]:
# 初始化模型
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')

#融合四个模型
model_plus = StackingRegressor(
                        regressors=[svr_lin, lr, ridge], 
                        meta_regressor=svr_rbf
                        )

In [9]:
names = ["LinearRegression", "svr_lin", "ridge", "svr_rbf", "model_plus"]
model = [lr, svr_lin, ridge, svr_rbf, model_plus]
for i in range(0, 5):
    model[i].fit(X2_train, y2_train)
    y2_pred = model[i].predict(X2_test)
    print(names[i], "%.5f" % mean_squared_error(y2_test, y2_pred))

LinearRegression 33.44898
svr_lin 39.41513
ridge 34.23161
svr_rbf 78.47805
model_plus 32.15039


e:\Anaconda\envs\ml\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
e:\Anaconda\envs\ml\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
e:\Anaconda\envs\ml\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
